<a href="https://colab.research.google.com/github/vasanramani/Great-Learning/blob/main/AppleHBR/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Install required libraries
!pip install -q langchain_community \
              langchain \
              langchain_openai \
              langchain-chroma \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              datasets==4.0.0 \
              evaluate==0.4.5

In [48]:
# Import core libraries
import os
import json
import requests
import chromadb

# Import libraries for working with PDFs and OpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
from IPython.display import Markdown, display

# Import libraries for processing dataframes and text
import tiktoken
import pandas as pd

# Import LangChain components for data loading, chunking, embedding, and vector DBs
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from datasets import Dataset
from langchain_openai import ChatOpenAI

modelToUse="gpt-4o-mini"
maxTokens=500
temp=0.2
topP=0.95

In [49]:
# Display Markdown formatted string in the notebook output.
# This function takes a string as input and uses IPython.display.Markdown
# to render it as Markdown, providing a more visually appealing output.
def printmd(string):
    display(Markdown(string))

In [50]:
# Identify the environment to switch the file paths either to use Google Drive or Local
import sys
# Check if the operating system is Windows
if sys.platform.startswith('win'):
    printmd("Running on Windows")
    resourcePath=""
else:
    # Assume it's a Linux-based environment, likely Google Colab
    printmd("Running on Linux most likely Colab")
    # Mount Google Drive for Colab environments to access files
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    # Define the base path for resources in Google Drive
    resourcePath="/content/drive/MyDrive/ColabAssignments/AppleHBR/"

Running on Windows

In [51]:
# Load Configuration for OPEN API
file_name = resourcePath + "config.json"
with open(file_name, 'r') as file:
    config = json.load(file)
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")

# Store API credentials in environment variables for use by LangChain and OpenAI clients
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE

In [52]:
# Initialize OpenAI client
openAIClient = OpenAI()

In [53]:
def getLLMResponse(userPrompt):
    # Access global variables for model configuration (model, max tokens, temperature, top_p)
    global modelToUse, maxTokens, temp, topP
    # Call the OpenAI API to get a chat completion
    completion = openAIClient.chat.completions.create(
        model=modelToUse, # Specify the AI model to use for the completion
        messages=[
            # Define the user's message to the AI model
            {"role": "user", "content": userPrompt}
        ],
        max_tokens=maxTokens, # Set the maximum number of tokens in the response
        temperature=temp,     # Control the randomness of the output (lower = more deterministic)
        top_p=topP            # Control diversity via nucleus sampling (fraction of tokens to consider)
    )
    # Extract and return the content of the AI's response
    return completion.choices[0].message.content

In [54]:
def getPromptedResponse(userPrompt):
    # Access global variables for model configuration
    global modelToUse, maxTokens, temp, topP, sysPromptDefault
    # Call the OpenAI API to get a chat completion with a system prompt for guidance
    completion = openAIClient.chat.completions.create(
        model=modelToUse, # Specify the AI model to use for the completion
        messages=[
            {"role": "system", "content": sysPromptDefault}, # Define the system's role and guiding prompt
            {"role": "user", "content": userPrompt}     # Define the user's message to the AI model
        ],
        max_tokens=maxTokens, # Set the maximum number of tokens in the response
        temperature=temp,     # Control the randomness of the output
        top_p=topP            # Control diversity via nucleus sampling
    )
    # Extract and return the content of the AI's response
    return completion.choices[0].message.content

In [55]:
# Load the PDF from the local location and verify the content
pdfPath = resourcePath + "HBR_How_Apple_Is_Organized_For_Innovation.pdf"
pdfLoader = PyMuPDFLoader(pdfPath)
pdfContent = pdfLoader.load()

In [56]:
# Split the text as multiple chunks
textSplitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=2000, #Complete the code to define the chunk size
    chunk_overlap= 200 #Complete the code to define the chunk overlap
)
pdfChunks = pdfLoader.load_and_split(textSplitter)
pdfChunksLength = len(pdfChunks)
printmd(f"The PDF File is split into {pdfChunksLength} Chunks")

The PDF File is split into 11 Chunks

In [57]:
# Initialize the Embeddings and verify if it loaded properly and it is of the same size
embeddingModel = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE
)
embedding0 = embeddingModel.embed_query(pdfChunks[0].page_content)
embedding1 = embeddingModel.embed_query(pdfChunks[1].page_content)

isEmbeddingSameSize = len(embedding0) == len(embedding1)
printmd(f"Dimension of the embedding vector is {len(embedding0)}")
printmd(f"Both Embeddings are {"SAME" if isEmbeddingSameSize else "DIFFERENT"} size")

Dimension of the embedding vector is 1536

Both Embeddings are SAME size

In [58]:
# Create the vector database from the chunks
vectorDBDir="/content/vectordb"
if not os.path.exists(vectorDBDir):
    os.makedirs(vectorDBDir)
    printmd("Vector DB Directory is created")
vectorDB = Chroma.from_documents(
    documents=pdfChunks,
    embedding=embeddingModel,
    persist_directory=vectorDBDir
)
printmd("Vector DB contents is created")

Vector DB contents is created

In [59]:
# Load the vector DB
vectorDBStore = Chroma(
    embedding_function=embeddingModel,
    persist_directory=vectorDBDir
)
vectorDBStore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000020F5B01BED0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000020F5BCC4910>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [60]:
vectorDBStore.similarity_search("author", k=2)

[Document(id='626cbe72-2151-47b1-a498-b01502eea777', metadata={'creationDate': "D:20201005141842-04'00'", 'total_pages': 11, 'modDate': 'D:20201201183749Z', 'title': '', 'file_path': 'HBR_How_Apple_Is_Organized_For_Innovation.pdf', 'subject': '', 'trapped': '', 'producer': 'Adobe PDF Library 15.0 (via http://bfo.com/products/pdf?version=2.23.5-r33279)', 'format': 'PDF 1.6', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'author': '', 'encryption': 'Standard V2 R3 128-bit RC4', 'source': 'HBR_How_Apple_Is_Organized_For_Innovation.pdf', 'page': 2, 'creationdate': '2020-10-05T14:18:42-04:00', 'keywords': '', 'moddate': '2020-12-01T18:37:49+00:00'}, page_content='PHOTOGRAPHER\u2002MIKAEL JANSSON\nHow Apple Is \nOrganized \nfor Innovation\nIt’s about experts \nleading experts.\nORGANIZATIONAL \nCULTURE\nJoel M. \nPodolny\nDean, Apple \nUniversity\nMorten T. \nHansen\nFaculty, Apple \nUniversity\nAUTHORS\nFOR ARTICLE REPRINTS CALL 800-988-0886 OR 617-783-7500, OR VISIT HBR.ORG\nHarvard Busine

In [61]:
# Initialize a retriever from the Chroma vector store
# This retriever will be used to fetch relevant document chunks based on a query.
vectorDBRetriever = vectorDBStore.as_retriever(
    search_type="similarity", # Specify that the search type should be 'similarity' to find semantically similar documents
    search_kwargs={"k": 2}    # Configure the search to return the top 2 most similar document chunk
)

In [62]:
def getRAGResponse(user_input,k=3):
    global qaSystemMsgTemplate, qaUserMsgTemplate, modelToUse, maxTokens, temp, topP
    # Retrieve relevant document chunks using the vector database retriever
    matchingChunks = vectorDBRetriever.invoke(user_input)
    # Extract the page content from the retrieved document chunks
    contexts = [d.page_content for d in matchingChunks]
    # Combine the extracted contexts into a single complete context string
    completeContext = ". ".join(contexts)
    # Populate the user message template with the retrieved context and user's question
    userMessage = qaUserMsgTemplate.replace('{context}', completeContext)
    userMessage = userMessage.replace('{question}', user_input)
    # Generate the response using the OpenAI client
    try:
        response = openAIClient.chat.completions.create(
        model=modelToUse,   # Specify the model to be used for completion
        messages=[
            {"role": "system", "content": ""}, # Provide the system prompt template to guide the AI's response
            {"role": "user", "content": userMessage}
        ],
        max_tokens=maxTokens,
        temperature=temp,
        top_p=topP
        )
        # Extract and print the generated text from the response
        response = response.choices[0].message.content.strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

In [63]:
def getResponse(user_question):
    global sysPromptDefault # Ensure global access if not defined in function scope

    printmd(f"## Question: {user_question}")

    printmd("### LLM Response:")
    llm_response = getLLMResponse(user_question)
    printmd(llm_response)
    
    printmd("### Prompted Response:")
    prompted_response = getPromptedResponse(user_question)
    printmd(prompted_response)

    printmd("### RAG Response:")
    rag_response = getRAGResponse(user_question)
    printmd(rag_response)

    printmd("\n---\n") # Separator for clarity

In [64]:
# Prompt used to guide the AI's responses

sysPromptDefault="""
You are an AI assistant developed to provide summaries and insights from research papers to support researchers 
and decision-makers in quickly understanding key findings and implications.

Your primary goal is to streamline the literature review process by:
- Extracting accurate and concise summaries from research papers.
- Identifying key metadata such as authors, publication year, research domain, and methodology.
- Highlighting emerging trends, insights, and innovations across related works.

When responding to the user's question, ensure that your answer is:
- Maintain factual accuracy and clarity at all times.
- Present insights in a structured, business-relevant, and easy-to-understand format.
- Avoid speculation or assumptions beyond the provided research content.
- If a query requires information not available in the given papers, acknowledge the limitation instead of inferring.

The objective is to help researchers and decision-makers quickly discover relevant information, reduce manual 
review time, and accelerate data-driven innovation.
"""

qaSystemMsgTemplate="""
You are an AI assistant designed to answer questions based on the following context extracted from a research paper. 

Your primary goal is to streamline the literature review process by:
- Extracting accurate and concise summaries from research papers.
- Identifying key metadata such as authors, publication year, research domain, and methodology.
- Highlighting emerging trends, insights, and innovations across related works.
- The task is to user questions by leveraging the provided context.

###Context: The context contains references to specify the detail and page numbers from the research paper. Use this 
context to provide accurate and concise answers to the user's questions.
{context} 
When responding to the user's question, ensure that your answer is:
- Accurate and directly supported by the provided context.
- Clear and concise, avoiding unnecessary information.
- Structured in a way that is easy to understand, using bullet points or numbered lists if appropriate
- If the context does not contain enough information to answer the question, respond with "The provided context does not contain sufficient information to answer this question."
"""

qaUserMsgTemplate="""
### Context: {context} $$$$ 
### Question: {question}
"""

In [65]:
# Below is the call to the getResponse method with the three different questions and different system prompts.
# The output is documented as a well formatted markdown document.
getResponse("Who are the authors of this article and who published this article?")

## Question: Who are the authors of this article and who published this article?

### LLM Response:

I'm sorry, but I cannot access external content, including specific articles, to provide information about authors or publishers. If you can provide the title or some details about the article, I may be able to help you with general information or context.

### Prompted Response:

Please provide the title or any specific details about the article you are referring to, so I can assist you in identifying the authors and the publisher.

### RAG Response:

The article is published by Harvard Business Review. However, the authors of the article are not mentioned in the provided context.


---


In [66]:
getResponse("List down the three leadership characteristics in bulleted points and explain each one of the characteristics under two lines.")

## Question: List down the three leadership characteristics in bulleted points and explain each one of the characteristics under two lines.

### LLM Response:

- **Vision**: Effective leaders possess a clear and compelling vision for the future, inspiring others to work towards common goals. This forward-thinking perspective helps guide decision-making and motivates team members.

- **Empathy**: Strong leaders demonstrate empathy by understanding and valuing the feelings and perspectives of their team members. This fosters trust, enhances communication, and creates a supportive work environment.

- **Integrity**: Leaders with integrity act consistently with their values and principles, earning the respect and trust of their followers. This characteristic promotes accountability and sets a positive example for others to follow.

### Prompted Response:

Here are three key leadership characteristics:

- **Visionary Thinking**: Effective leaders possess the ability to create and communicate a clear vision for the future, inspiring their team to work towards common goals and navigate challenges.

- **Emotional Intelligence**: Leaders with high emotional intelligence can understand and manage their own emotions while empathizing with others, fostering strong relationships and a positive team environment.

- **Decisiveness**: Strong leaders are able to make timely and informed decisions, even in uncertain situations, demonstrating confidence and accountability that instills trust within their team.

### RAG Response:

- **Deep Expertise**: Apple managers are expected to have specialized knowledge in their respective fields, allowing them to engage meaningfully in their functions. This focus on expertise ensures that decisions are made by those who truly understand the technical and operational nuances of their areas.

- **Immersion in Details**: Leaders at Apple are required to be deeply involved in the specifics of their functions, which fosters a thorough understanding of the challenges and opportunities present. This immersion enables them to make informed decisions that align with the company's strategic goals.

- **Collaborative Debate**: Apple encourages a culture of open discussion among leaders from different functions, promoting diverse perspectives in decision-making. This collaborative approach ensures that decisions are well-rounded and consider the implications across various areas of the organization.


---


In [67]:
getResponse("Can you explain specific examples from the article where Apple's approach to leadership has led to successful innovations?")

## Question: Can you explain specific examples from the article where Apple's approach to leadership has led to successful innovations?

### LLM Response:

While I don't have access to specific articles, I can provide a general overview of how Apple's approach to leadership has led to successful innovations based on widely known practices and historical examples.

1. **Visionary Leadership**: Under Steve Jobs, Apple focused on a clear vision of creating products that combined technology with art. This vision led to the development of iconic products like the iPhone, which revolutionized the smartphone industry by integrating a user-friendly interface with powerful capabilities.

2. **Emphasis on Design**: Apple's leadership has always prioritized design and user experience. The introduction of the iMac in 1998 showcased this approach, with its unique aesthetics and intuitive interface, which helped to revitalize the company's brand and market presence.

3. **Cross-Functional Collaboration**: Apple's leadership encourages collaboration across different departments. For instance, the development of the Apple Watch involved teams from hardware, software, and health sectors working together, resulting in a product that not only functions as a smartwatch but also emphasizes health and fitness tracking.

4. **Focus on Ecosystem**: Apple's leadership has strategically built an ecosystem of products and services that work seamlessly together. The introduction of services like iCloud and Apple Music has enhanced user loyalty and created a cohesive experience across devices, driving innovation in how users interact with technology.

5. **Risk-Taking and Innovation**: Apple's leadership has not shied away from taking risks. The launch of the iPad in 2010 was a bold move into a new product category. Despite skepticism, the iPad became a significant success, creating a new market for tablets.

6. **Customer-Centric Approach**: Apple's leadership places a strong emphasis on understanding customer needs. The development of features like Face ID and the emphasis on privacy in their products reflect a commitment to enhancing user experience and security, which has resonated well with consumers.

These examples illustrate how Apple's leadership style, characterized by a strong vision, design focus, collaboration, ecosystem thinking, risk-taking, and customer orientation, has driven successful innovations over the years.

### Prompted Response:

I currently do not have access to specific articles or their content. However, I can provide a general overview of Apple's approach to leadership and how it has historically led to successful innovations.

1. **Visionary Leadership**: Steve Jobs, co-founder of Apple, was known for his visionary leadership style. His ability to foresee market trends and consumer needs led to the development of groundbreaking products like the iPhone and iPad. His focus on design and user experience set Apple apart from competitors.

2. **Cross-Functional Collaboration**: Apple's leadership encourages collaboration across different departments, such as design, engineering, and marketing. This approach was evident in the development of the Apple Watch, where input from various teams resulted in a product that combined technology with health and fitness features.

3. **Emphasis on Design and User Experience**: Apple's leadership prioritizes aesthetics and usability, which has been a key factor in the success of products like the MacBook and iPhone. The integration of hardware and software design has created a seamless user experience that fosters customer loyalty.

4. **Risk-Taking and Innovation Culture**: Apple's leadership fosters a culture that embraces risk-taking and innovation. The introduction of the App Store transformed the software distribution model and created a new ecosystem for developers, significantly contributing to Apple's revenue growth.

5. **Focus on Privacy and Security**: Under Tim Cook's leadership, Apple has emphasized user privacy and security, differentiating itself from competitors. This focus has led to innovations such as advanced encryption and privacy features in iOS, enhancing customer trust and brand loyalty.

If you have a specific article in mind, please provide its details, and I can help summarize its findings related to Apple's leadership and innovations.

### RAG Response:

Certainly! The article highlights several specific examples of how Apple's approach to leadership—characterized by deep expertise, immersion in details, and collaborative debate—has led to successful innovations:

1. **Roger Rosner and Software Applications**: Roger Rosner, who heads Apple’s software application business, exemplifies the principle of having experts lead experts. His background in electrical engineering and extensive experience in software engineering allowed him to effectively guide teams working on productivity apps like Pages, Numbers, and Keynote. His leadership has fostered an environment where specialists can collaborate and innovate, resulting in high-quality applications that enhance user productivity.

2. **Camera Hardware Technology Team**: Apple has over 600 experts focused on camera hardware technology, led by Graham Townsend, a camera expert. This functional organization allows these specialists to work closely together rather than being scattered across different product lines. By concentrating their expertise, they can solve problems more effectively and innovate in camera technology, which is crucial for products like the iPhone and iPad. This collaborative environment has led to advancements in camera capabilities, such as improved image quality and innovative features.

3. **Attention to Detail in Product Design**: The article discusses Apple's meticulous attention to the design of product features, such as the rounded corners of devices. Apple leaders insist on continuous curves, known as "squircles," instead of the standard abrupt arcs. This focus on detail not only enhances the aesthetic appeal of products but also improves functionality, such as reducing light reflection. The commitment to precise manufacturing tolerances ensures that millions of products can be produced consistently, leading to high-quality outcomes that resonate with consumers.

4. **Collaborative Development of Dual-Lens Camera**: The development of the dual-lens camera with portrait mode required collaboration among multiple specialist teams. This cross-functional teamwork exemplifies Apple's culture of collaborative debate, where experts from various fields come together to innovate and refine product features. The successful implementation of this technology has significantly enhanced the photography capabilities of Apple devices, contributing to their competitive edge in the market.

These examples illustrate how Apple's leadership philosophy—emphasizing expertise, detail orientation, and collaboration—has been instrumental in driving innovation and maintaining the company's reputation for high-quality products.


---


In [68]:
getResponse("What is an example of the attention to detail?")

## Question: What is an example of the attention to detail?

### LLM Response:

Attention to detail refers to the ability to notice and consider all the small elements of a task or project, ensuring accuracy and quality. An example of attention to detail can be seen in the work of a graphic designer. 

For instance, when creating a marketing brochure, a designer might:

1. **Color Consistency**: Ensure that the colors used in the text, images, and backgrounds are consistent with the brand's color palette.
2. **Font Alignment**: Carefully align text and images to create a balanced and professional layout, checking that margins and spacing are uniform.
3. **Proofreading**: Meticulously proofread the content for spelling and grammatical errors, as well as ensuring that all information is accurate and up-to-date.
4. **Image Quality**: Select high-resolution images that are relevant and appropriate for the target audience, ensuring they are not pixelated or distorted.
5. **Feedback Incorporation**: Actively seek and incorporate feedback from clients or team members, making adjustments to improve the overall design.

This level of attention to detail not only enhances the visual appeal of the brochure but also ensures that it effectively communicates the intended message and reflects positively on the brand.

### Prompted Response:

Attention to detail refers to the ability to notice and consider all aspects of a task or situation, ensuring accuracy and thoroughness. An example of attention to detail can be found in the field of quality control in manufacturing. 

For instance, a quality control inspector meticulously examines each product on the assembly line for defects. This includes checking for proper dimensions, surface finish, and functionality. By carefully documenting any discrepancies and ensuring that each product meets the specified standards, the inspector helps maintain high quality and consistency in the final output. 

This level of scrutiny not only prevents defective products from reaching customers but also enhances overall operational efficiency and customer satisfaction.

### RAG Response:

An example of Apple's attention to detail is their use of a "squircle" for rounding the corners of rectangular objects. Unlike the standard method, which uses an arc of a circle that can result in an abrupt transition in curvature, the squircle creates continuous curves. This design choice helps to produce softer highlights by minimizing light reflection, demonstrating Apple's commitment to meticulous design and user experience.


---


In [69]:
# Generate actionable insights and business recommendations using the RAG pipeline
insights_prompt = (
    "Based on the article 'How Apple Is Organized for Innovation', generate 5 concise, actionable insights "
    "that product and engineering leaders can apply. For each insight provide a one-line action and a one-sentence rationale."
)

recommendations_prompt = (
    "Based on the same article, produce 7 concrete business recommendations for senior leadership "
    "and org design to improve innovation outcomes. For each recommendation provide a short rationale and a suggested first step."
)

# Get RAG-backed responses
actionable_insights = getRAGResponse(insights_prompt)
business_recommendations = getRAGResponse(recommendations_prompt)

# Display in notebook
printmd("## Actionable Insights")
printmd(actionable_insights)

printmd("## Business Recommendations")
printmd(business_recommendations)

# Persist results to a JSON file for reuse
output = {
    "actionable_insights": actionable_insights,
    "business_recommendations": business_recommendations
}

## Actionable Insights

1. **Foster Cross-Disciplinary Collaboration**  
   Action: Create cross-functional teams that bring together experts from various fields.  
   Rationale: This approach encourages diverse perspectives and innovative solutions, mirroring Apple's model of integrating expertise across disciplines.

2. **Empower Expert Leadership**  
   Action: Allow subject matter experts to lead projects within their domains.  
   Rationale: When experts guide their peers, it enhances decision-making and drives innovation, as seen in Apple's organizational structure.

3. **Encourage Iterative Feedback Loops**  
   Action: Implement regular feedback sessions throughout the development process.  
   Rationale: Continuous feedback fosters a culture of improvement and adaptability, crucial for innovation in fast-paced environments.

4. **Prioritize a Culture of Experimentation**  
   Action: Allocate resources for teams to test new ideas without the fear of failure.  
   Rationale: Embracing experimentation allows for creative risk-taking, which is essential for breakthrough innovations like those at Apple.

5. **Align Goals with Innovation Objectives**  
   Action: Ensure that team objectives are directly linked to innovation targets.  
   Rationale: Clear alignment between individual and organizational goals drives focus and motivation, facilitating a culture of innovation similar to Apple's.

## Business Recommendations

Based on the insights from the article "How Apple Is Organized for Innovation," here are seven concrete business recommendations for senior leadership and organizational design to improve innovation outcomes:

1. **Foster a Culture of Collaboration Among Experts**
   - **Rationale:** Innovation thrives in environments where experts from different fields collaborate, share ideas, and challenge each other’s thinking. This cross-pollination can lead to breakthrough ideas and solutions.
   - **First Step:** Create interdisciplinary project teams that bring together experts from various departments to work on specific innovation initiatives.

2. **Implement a Flat Organizational Structure**
   - **Rationale:** A flatter hierarchy can reduce bureaucratic barriers, allowing for faster decision-making and empowering employees to take ownership of their projects.
   - **First Step:** Conduct an organizational audit to identify layers of management that can be streamlined and develop a plan to flatten the structure where feasible.

3. **Encourage Risk-Taking and Accept Failure**
   - **Rationale:** A culture that embraces experimentation and tolerates failure can lead to more innovative ideas, as employees feel safe to take risks without fear of repercussions.
   - **First Step:** Establish a “failure fund” that rewards teams for taking calculated risks, regardless of the outcome, and share stories of both successes and failures to normalize the learning process.

4. **Invest in Continuous Learning and Development**
   - **Rationale:** Keeping employees’ skills up-to-date and exposing them to new ideas and technologies can enhance their ability to innovate.
   - **First Step:** Develop a comprehensive training program that includes workshops, seminars, and access to online courses focused on emerging trends and technologies relevant to the business.

5. **Create Dedicated Innovation Labs**
   - **Rationale:** Innovation labs provide a dedicated space for experimentation and creativity, free from the constraints of day-to-day operations.
   - **First Step:** Identify a physical space within the organization that can be transformed into an innovation lab and allocate resources for tools and materials needed for experimentation.

6. **Establish Clear Innovation Metrics**
   - **Rationale:** Measuring innovation outcomes can help organizations understand what works and what doesn’t, allowing for better strategic decisions moving forward.
   - **First Step:** Define key performance indicators (KPIs) related to innovation, such as the number of new products launched, time to market, and revenue generated from new offerings, and implement a tracking system.

7. **Promote Open Communication and Idea Sharing**
   - **Rationale:** Open lines